# IBM Watson Annotator for Clinical Data Python SDK Example

### Intro
This is a short ACD sample application that demonstrates how to create a client to communicate with ACD, invoke ACD to analyze some text using a persisted flow, and retrieve annotation fields from the response.  In this example, the client config parameters are stored in a separate file called `config.ini` and Python's configparser is used to parse the file.  The format of `config.ini` is:

```
[settings]
base_url = your ACD service instance URL, e.g. https://us-south.wh-acd.cloud.ibm.com/wh-acd/api
key = API Key of your ACD service instance
version = ACD REST API version date, e.g. 2021-04-15
flow = wh_acd.ibm_clinical_insights_v1.0_standard_flow
```

### Install SDK

In [1]:
!pip install ibm-whcs-sdk

### Client Setup

In [2]:
# A simple example to demonstrate how the SDK could be used.

from ibm_cloud_sdk_core.authenticators.iam_authenticator import IAMAuthenticator
import ibm_whcs_sdk.annotator_for_clinical_data as wh
import configparser

CONFIG = configparser.RawConfigParser()
CONFIG.read('config.ini')

APIKEY = CONFIG.get('settings', 'key')
VERSION = CONFIG.get('settings', 'version')
URL = CONFIG.get('settings', 'base_url')
FLOW = CONFIG.get('settings', 'flow')

acd = wh.AnnotatorForClinicalDataV1(authenticator=IAMAuthenticator(apikey=APIKEY),
                                    version=VERSION)
acd.set_service_url(URL)
print("ACD Client is Ready!")

ACD Client is Ready!


### Invoke ACD's Analyze_with_flow API then parse the response

In [9]:
TEXT = "Patient has lung cancer, but did not smoke. CT scan shows tumor in left lung. She may consider chemotherapy as part of a treatment plan."
try:
    result = acd.analyze_with_flow(FLOW, TEXT)
    
    print("-------------------------------------------------")
    print("Text:")
    print(TEXT)
    print("-------------------------------------------------")
    print("Patient Procedures")
    if result.procedure_ind is not None:
        for item in result.procedure_ind:
            print("..",item.procedure_normalized_name)
            print("    Notes:")
            if item.insight_model_data.procedure is not None:
                proc = item.insight_model_data.procedure
                if proc.usage is not None:
                    if proc.usage.explicit_score > 0.7:
                        print("    - Completed (confidence score", proc.usage.explicit_score, ")")
                    elif proc.usage.pending_score > 0.7:
                        print("    - Scheduled/Recommended (confidence score", proc.usage.pending_score, ")")
                    elif proc.usage.discussed_score > 0.7:
                        print("    - Discussed/Other (confidence score", proc.usage.discussed_score, ")")
                if proc.type is not None:
                    if proc.type.device_score > 0.7:
                        print("    - Implanted device (confidence score", proc.type.device_score, ")")
                    elif proc.type.material_score > 0.7:
                        print("    - Material or graft (confidence score", proc.type.material_score, ")")
                    elif proc.type.medication_score > 0.7:
                        print("    - Medication (confidence score", proc.type.medication_score, ")")
                    elif proc.type.condition_management_score > 0.7:
                        print("    - Long term condition (confidence score", proc.type.condition_management_score, ")")
                    elif proc.type.procedure_score > 0.7:
                        print("    - Other procedure (confidence score", proc.type.procedure_score, ")")
            if item.insight_model_data.normality is not None:
                norm = item.insight_model_data.normality
                if norm.usage is not None:
                    if norm.usage.normal_score > 0.7:
                        print("    -- Normal (confidence score", norm.usage.normal_score, ")")
                        if norm.evidence is not None:
                            for evi_entry in norm.evidence:
                                print("         Evidence: ", evi_entry.covered_text)
                    elif norm.usage.abnormal_score > 0.7:
                        print("    ** Abnormal (confidence score", norm.usage.abnormal_score, ")")
                        if norm.evidence is not None:
                            for evi_entry in norm.evidence:
                                print("         Evidence: ", evi_entry.covered_text)
            print("")
    print("-------------------------------------------------")
    print("Patient Symptoms and Diseases")
    if result.symptom_disease_ind is not None:
        for item in result.symptom_disease_ind:
            print("..",item.symptom_disease_normalized_name, "(", item.covered_text, ")")
            print("    Notes:")
            if item.snomed_concept_id is not None:
                print("      - SNOMED", item.snomed_concept_id)
            if item.icd10_code is not None:
                print("      - ICD10", item.icd10_code)
            if item.icd9_code is not None:
                print("      - ICD9", item.icd9_code)
            if item.insight_model_data is not None:
                if item.insight_model_data.diagnosis is not None:
                    diag = item.insight_model_data.diagnosis
                    if diag.usage is not None:
                        if diag.usage.explicit_score > 0.7:
                            print("    - Applies to patient (confidence score", diag.usage.explicit_score, ")")
                        elif diag.usage.patient_reported_score > 0.7:
                            print("    - Patient reported (confidence score", diag.usage.patient_reported_score, ")")
                        elif diag.usage.discussed_score > 0.7:
                            print("    - Other (confidence score", diag.usage.discussed_score, ")")
            print("")

except wh.ACDException as ex:
    print ("Error Occurred:  Code: ", ex.code, " Message: ", ex.message, " CorrelationId: ", ex.correlation_id, " Additional Error Details: ",  ex.err_description)

-------------------------------------------------
Text
Patient has lung cancer, but did not smoke. CT scan shows tumor in left lung. She may consider chemotherapy as part of a treatment plan.
-------------------------------------------------
Patient Procedures
.. c.a.t. scan nos
    Notes:
    - Completed (confidence score 0.976 )
    - Other procedure (confidence score 1.0 )
    ** Abnormal (confidence score 0.994 )
         Evidence:  shows tumor in left lung

.. chemotherapy
    Notes:
    - Discussed/Other (confidence score 0.944 )
    - Medication (confidence score 1.0 )

-------------------------------------------------
Patient Symptoms and Diseases
.. primary malignant neoplasm of lung ( lung cancer )
    Notes:
      - SNOMED 93880001
      - ICD10 C34.90,C34.9
      - ICD9 162.9
    - Applies to patient (confidence score 0.999 )

.. neoplasm ( tumor )
    Notes:
      - SNOMED 108369006

